In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import glob
import os
import numpy as np

# Loads each whole scenario as a sheet

In [2]:
start_folder = 'AllCSVs'
csvs_scen = glob.glob(r'{}\*.csv'.format(start_folder))
df_scen = []

for c in csvs_scen:
    df = pd.read_csv(c)
    df_scen.append(df)
print(len(df_scen))
# df_list[0]

223


In [3]:
df = df_scen[10].iloc[0:20,:].drop(['type','spd_noise0','spd_noise1','spd_noise2' ,'pos_noise0' ,'pos_noise1' ,'pos_noise2', 'spd2', 'pos2'], axis=1)
df

,time,sender,attackerType,messageID,pos0,pos1,spd0,spd1
0,10800.392785,13,0,38,3597.152086,5542.199221,-3.178366,38.798969
1,10800.437587,7,0,71,3609.398494,5446.803264,-3.534058,30.622234
2,10800.456150,31,0,104,3596.805861,5692.113417,1.282467,-29.864003
3,10800.582020,25,0,137,3597.770493,5764.800893,-0.020161,-34.568848
4,10800.670638,43,0,172,3822.892006,5266.806009,1.305362,-0.988730
5,10800.870012,19,0,193,3617.100034,5417.782781,4.937950,-39.877578
6,10800.902598,37,0,228,3592.547591,6044.106465,4.097281,38.748991
7,10801.392785,13,0,257,3593.982140,5580.895412,-3.166225,38.650764
8,10801.437587,7,0,284,3605.873370,5477.348089,-3.533871,30.620620
9,10801.456150,31,0,317,3598.091089,5662.185142,1.282719,-29.869871


In [5]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from scipy.spatial import distance

# Define a function to find the two closest rows above a given row based on A and B values
def find_closest_rows_above(df, row):
    # select a row to find closest rows to
    given_row = row

    # Find the index of the given row
    row_index = df.index.get_loc(given_row.name)

    # Slice the DataFrame to only consider rows above the given row
    above_df = df.iloc[:row_index]

    if len(above_df) == 0:
        return pd.DataFrame()


    # select the A and B columns of the DataFrame
    df_ab = above_df[['time', 'pos0', 'pos1']]

    # calculate the Euclidean distance between the given row and all other rows based on A and B values
    distances = distance.cdist(df_ab, [given_row[['time', 'pos0', 'pos1']]], metric='euclidean')

    # get the indices of rows sorted by distance
    sorted_indices = distances.argsort(axis=0).flatten()

    # select the N closest rows (excluding the given row itself)
    closest_indices = sorted_indices[0:3] # ** Number of closest cars we want to Keep **

    # get the closest rows as a DataFrame
    closest_rows = df.iloc[closest_indices]
    
    # Create a Boolean mask to select the rows that contain the value
    mask = closest_rows['sender'] == search_row.sender

    # Drop the rows that match the mask
    closest_rows.drop(closest_rows[mask].index, inplace=True)

    return closest_rows

for i in range(0, len(df)): #len(df)
    search_row = df.iloc[i] 
    print('------------------------------------------------------------------------')
    closest_rows = find_closest_rows_above(df, search_row)
    display(search_row.to_frame().T)
    display(closest_rows)

------------------------------------------------------------------------


,time,sender,attackerType,messageID,pos0,pos1,spd0,spd1
0,10800.392785,13.0,0.0,38.0,3597.152086,5542.199221,-3.178366,38.798969


""


------------------------------------------------------------------------


,time,sender,attackerType,messageID,pos0,pos1,spd0,spd1
1,10800.437587,7.0,0.0,71.0,3609.398494,5446.803264,-3.534058,30.622234


...

,time,sender,attackerType,messageID,pos0,pos1,spd0,spd1
18,10802.870012,19.0,0.0,573.0,3627.003043,5337.808701,4.940953,-39.901824


,time,sender,attackerType,messageID,pos0,pos1,spd0,spd1
1,10800.437587,7,0,71,3609.398494,5446.803264,-3.534058,30.622234


------------------------------------------------------------------------


,time,sender,attackerType,messageID,pos0,pos1,spd0,spd1
19,10803.392785,13.0,0.0,603.0,3588.263022,5658.409084,-1.620634,38.77584


,time,sender,attackerType,messageID,pos0,pos1,spd0,spd1
9,10801.45615,31,0,317,3598.091089,5662.185142,1.282719,-29.869871
15,10802.45615,31,0,493,3599.868069,5632.305432,2.507947,-29.791207
2,10800.45615,31,0,104,3596.805861,5692.113417,1.282467,-29.864003


# ------------------------------------------Template Code-----------------------------------------

In [ ]:
import pandas as pd
from scipy.spatial import distance

# create a sample DataFrame
df = pd.DataFrame({
    'A': [1, 2, 3, 4, 4,6,7,8,9],
    'B': [2, 4, 6, 8, 7,3,7,3,8],
    'C': [3, 6, 9, 12, 3,2,9,6,4]
})

# select a row to find closest rows to
given_row = df.iloc[4]

# Find the index of the given row
row_index = df.index.get_loc(given_row.name)

# Slice the DataFrame to only consider rows above the given row
above_df = df.iloc[:row_index]

# if len(above_df) == 0:
#     return pd.DataFrame()


# select the A and B columns of the DataFrame
df_ab = above_df[['A', 'B']]

# calculate the Euclidean distance between the given row and all other rows based on A and B values
distances = distance.cdist(df_ab, [given_row[['A', 'B']]], metric='euclidean')

# get the indices of rows sorted by distance
sorted_indices = distances.argsort(axis=0).flatten()

# select the two closest rows (excluding the given row itself)
closest_indices = sorted_indices[0:3]

# get the closest rows as a DataFrame
closest_rows = df.iloc[closest_indices]

print(closest_rows)

In [ ]:
import pandas as pd
from scipy.spatial.distance import cdist

# Generate a sample DataFrame
df = pd.DataFrame({'A': [1, 1, 1, 4, 1],
                   'B': [6, 2, 2, 9, 2],
                   'C': [11, 3, 3, 14, 3]})

# Define a function to find the two closest rows above a given row based on A and B values
def find_closest_rows_above(df, row):
    # Find the index of the given row
    row_index = df.index.get_loc(row.name)
    
    # Slice the DataFrame to only consider rows above the given row
    above_df = df.iloc[:row_index]
    
    if len(above_df) == 0:
        return pd.DataFrame()
    
    # Compute the distances between the given row and all rows above it in the DataFrame based on A and B values
    distances = cdist(above_df[['A', 'B']].values, row[['A', 'B']].values.reshape(1, -1))
    
    # Sort the distances in ascending order and retrieve the index of the closest row
    closest_index = distances.argmin(axis=0)[0]
    
    # Use the index to retrieve the closest row from the DataFrame
    closest_row = above_df.iloc[closest_index]
    
    # Find the index of the second closest row
    second_closest_index = (distances != distances[closest_index]).argmax(axis=0)[0]
    
    # Use the index to retrieve the second closest row from the DataFrame
    second_closest_row = above_df.iloc[second_closest_index]
    
    return pd.concat([closest_row, second_closest_row], axis=1).T

# Define a sample row to search for
search_row = df.iloc[2] 
# For i in range(0, len(df)): 
#     search_row = df.iloc[i]
# Append the closest_rows to a new dataframe and then glue the 2 dataframes together at the end




# Find the two closest rows above the search row based on A and B values
closest_rows = find_closest_rows_above(df, search_row)

print(closest_rows)

In [ ]:
df

In [ ]:
import pandas as pd

# create a DataFrame
df = pd.DataFrame({'pos1': [25, 30, 35],
                   'pos2': [25, 30, 35],
                   'pos3': [25, 30, 35]})

# create a new DataFrame with the new columns
closest_rows = pd.DataFrame({'pos1DIFF': [50000, 60000, 70000],
                         'pos2DIFF': [1, 2, 3]})

# concatenate the new DataFrame to the existing one
result = pd.concat([df, closest_rows], axis=1)

print(result)

In [ ]:
X = []
y = []
for i in df_normal:
    X.append(np.asarray(df_normal[0].iloc[0:10,2:]))
    y.append(np.asarray(df_normal[0].iloc[10,2:]))
X = np.asarray(X)
y = np.asarray(y)
np.save(r'C:\Users\aidan\Desktop\School\5G_Networks\Final_Project\{}\X.npy'.format(start_folder), X)
np.save(r'C:\Users\aidan\Desktop\School\5G_Networks\Final_Project\{}\y.npy'.format(start_folder), y)